# HuggingFace Chat Target Testing

The goal is to optimize the notebook demo with a small yet effective instruction model, or even a chat model, allowing users to experiment with different HuggingFace models. Instruction models can also function as chat models and vice versa, enabling flexible interactions and a variety of use cases.

## Key Notes:

1. **Additional Dependencies**:
   - Some models, such as `"princeton-nlp/Sheared-LLaMA-1.3B-ShareGPT"` and `"princeton-nlp/Sheared-LLaMA-2.7B-ShareGPT"`, require the `sentencepiece` library to be installed. If you encounter errors related to this, you can install `sentencepiece` via pip:

     ```bash
     pip install sentencepiece
     ```

2. **Access Requirements for Certain Models**:
   - Some models, like `"google/gemma-2b"`, need special access. You must request access on the model's Hugging Face page and agree to the "Terms of Use."
   - Ensure your Hugging Face account email matches any email provided on external sites (e.g., Meta) to avoid request denial.
   - After requesting, wait for approval from the model's authors.

3. **Issues with Some Model Outputs**:
   - During testing, it's been observed that some models may output responses that do not make sense or are not coherent. These models might require further adjustments, fine-tuning, or attention to their settings and prompts to improve their performance. It is advisable to take a closer look at these models and assess whether they need to be fixed or optimized for better results.

4. **Model Performance and Response Times**:
   - The tests were conducted using a CPU, and the following are the average response times for each model:
     - `microsoft/Phi-3-mini-4k-instruct`: 4.89 seconds
     - `HuggingFaceTB/SmolLM-135M-Instruct`: 3.09 seconds
     - `HuggingFaceTB/SmolLM-360M-Instruct`: 3.31 seconds
     - `HuggingFaceTB/SmolLM-1.7B-Instruct`: 5.87 seconds
     - `Qwen/Qwen2-0.5B-Instruct`: 1.38 seconds
     - `Qwen/Qwen2-1.5B-Instruct`: 2.96 seconds
     - `stabilityai/stablelm-2-zephyr-1_6b`: 5.31 seconds
     - `stabilityai/stablelm-zephyr-3b`: 8.37 seconds
     - `google/gemma-2b`: 7.79 seconds
     - `TinyLlama/TinyLlama-1.1B-Chat-v1.0`: 4.03 seconds
     - `princeton-nlp/Sheared-LLaMA-1.3B-ShareGPT`: 3.36 seconds
     - `princeton-nlp/Sheared-LLaMA-2.7B-ShareGPT`: 8.29 seconds


In [1]:
import time
from pyrit.prompt_target import HuggingFaceChatTarget  
from pyrit.orchestrator import PromptSendingOrchestrator

# List of models to iterate through
model_list = [
    "microsoft/Phi-3-mini-4k-instruct",
    "HuggingFaceTB/SmolLM-135M-Instruct",
    "HuggingFaceTB/SmolLM-360M-Instruct",
    "HuggingFaceTB/SmolLM-1.7B-Instruct",
    "Qwen/Qwen2-0.5B-Instruct",
    "Qwen/Qwen2-1.5B-Instruct",
    "stabilityai/stablelm-2-zephyr-1_6b",
    "stabilityai/stablelm-zephyr-3b",
    "google/gemma-2b",
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    "princeton-nlp/Sheared-LLaMA-1.3B-ShareGPT",
    "princeton-nlp/Sheared-LLaMA-2.7B-ShareGPT"
]
   
# List of prompts to send
prompt_list = [
        "What is 3*3?",
        "What is 4*4?",
        "What is 5*5?",
        "What is 6*6?",
        "What is 7*7?"
    ]


# Dictionary to store average response times
model_times = {}
    
for model_id in model_list:
    print(f"Running model: {model_id}")
        
    try:
        # Initialize HuggingFaceChatTarget with the current model
        target = HuggingFaceChatTarget(
            model_id=model_id,          # Use the desired model ID
            use_cuda=False,             # Set to True if using CUDA
            tensor_format="pt",
            verbose=False,              # Set to True for detailed logs
            max_new_tokens=30
        )
            
        # Initialize the orchestrator
        orchestrator = PromptSendingOrchestrator(
            prompt_target=target,
            verbose=False
        )
            
        # Record start time
        start_time = time.time()
            
        # Send prompts asynchronously
        responses = await orchestrator.send_prompts_async(prompt_list=prompt_list)
            
        # Record end time
        end_time = time.time()
            
        # Calculate total and average response time
        total_time = end_time - start_time
        avg_time = total_time / len(prompt_list)
        model_times[model_id] = avg_time
            
        print(f"Average response time for {model_id}: {avg_time:.2f} seconds\n")
            
        # Print the conversations
        orchestrator.print_conversations()
        print("-" * 50)
        
    except Exception as e:
        print(f"An error occurred with model {model_id}: {e}\n")
        model_times[model_id] = None
        continue
    
# Print all model average times
print("Model Average Response Times:")
for model_id, avg_time in model_times.items():
    if avg_time is not None:
        print(f"{model_id}: {avg_time:.2f} seconds")
    else:
        print(f"{model_id}: Error occurred, no average time calculated.")

Running model: microsoft/Phi-3-mini-4k-instruct


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are not running the flash-attention implementation, expect numerical differences.


Average response time for microsoft/Phi-3-mini-4k-instruct: 4.89 seconds

Conversation ID: 59de0019-3cf2-44b6-bbdf-261c1d3fd211
user: What is 7*7?
assistant: What is 7*7? 7 multiplied by 7 equals 49.
Conversation ID: 721df2cd-d7d9-4a56-ae44-956b0fb3439e
user: What is 6*6?
assistant: What is 6*6? 6 multiplied by 6 equals 36.
Conversation ID: 73fa053f-b85b-406f-85ef-c60a280338f8
user: What is 5*5?
assistant: What is 5*5? 5 multiplied by 5 equals 25.
Conversation ID: 79c96c19-4256-41bb-8f65-92f67c279db8
user: What is 4*4?
assistant: What is 4*4? 4 multiplied by 4 equals 16.
Conversation ID: 902384b3-b1a7-4d28-a4ef-c69d541e63db
user: What is 3*3?
assistant: What is 3*3? 3 multiplied by 3 equals 9.
--------------------------------------------------
Running model: HuggingFaceTB/SmolLM-135M-Instruct
Average response time for HuggingFaceTB/SmolLM-135M-Instruct: 3.09 seconds

Conversation ID: 05168533-b80c-455e-ae99-aab48c88c23d
user: What is 4*4?
assistant: user
What is 4*4?
assistant
What a g

...

Average response time for google/gemma-2b: 7.87 seconds

Conversation ID: 1f592b01-a87c-4a16-94ac-5269dca38e70
user: What is 4*4?
assistant: <|im_start|>user
What is 4*4?<|im_end|>
<|im_start|>assistant
<|im_im|>im_im_im_im_im_im_im_im_im_im_im_im_
Conversation ID: 26c095a9-b05f-4236-b691-3aa93378f169
user: What is 6*6?
assistant: <|im_start|>user
What is 6*6?<|im_end|>
<|im_start|>assistant
<|im_end|>
<|im_start|>
<|im_end|>
<|im_start|>
<|
Conversation ID: 40eacc48-170f-4a35-a2a1-da784e43d2e4
user: What is 7*7?
assistant: <|im_start|>user
What is 7*7?<|im_end|>
<|im_start|>assistant
<|im_assistant|>
<|im_assistant|>
<|im_assistant|>
<|im_assistant|>
<|
Conversation ID: a09602fc-18c3-41b6-8823-8210e9980baa
user: What is 3*3?
assistant: <|im_start|>user
What is 3*3?<|im_end|>
<|im_start|>assistant
<|im_start|>
<|im_start|>
<|im_start|>
<|im_start|>
<|
Conversation ID: e40eeab4-bd7b-4dfd-a5eb-416da746d52f
user: What is 5*5?
assistant: <|im_start|>user
What is 5*5?<|im_end|>
<|im_start|>

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Average response time for princeton-nlp/Sheared-LLaMA-1.3B-ShareGPT: 3.17 seconds

Conversation ID: 06b1f37a-30ff-4592-8bf4-8429496e6bb4
user: What is 6*6?
assistant: [INST] What is 6*6? [/INST]

6*6 is equal to 6x6 = 66.
Conversation ID: 22efae3f-0bf8-4129-a150-9c77355415ea
user: What is 7*7?
assistant: [INST] What is 7*7? [/INST]

7*7 is equal to 7 x 7 = 77.
Conversation ID: 35813c4e-e90a-4f09-9934-e8732cd0d1f2
user: What is 4*4?
assistant: [INST] What is 4*4? [/INST]

4*4 is equal to 4 times 4, which is 16.
Conversation ID: 91ca9bfc-af37-4685-a18e-2255bb25448f
user: What is 5*5?
assistant: [INST] What is 5*5? [/INST]

I'm sorry, I'm not sure what you're asking. Could you please provide more context or clarify your question?
Conversation ID: d5c25d93-03e4-4e58-9508-7aed7e2f6cfb
user: What is 3*3?
assistant: [INST] What is 3*3? [/INST]

3*3 is equal to 3x3.
--------------------------------------------------
Running model: princeton-nlp/Sheared-LLaMA-2.7B-ShareGPT


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Average response time for princeton-nlp/Sheared-LLaMA-2.7B-ShareGPT: 8.54 seconds

Conversation ID: 21e5b6f2-abc3-4f76-abba-2b19a98fa28c
user: What is 4*4?
assistant: [INST] What is 4*4? [/INST]

[INST]What is 4^4? [/INST]

[INST]What is 4^4? [/
Conversation ID: 5768ec92-b036-4b5b-a518-f3eeea7bc18a
user: What is 5*5?
assistant: [INST] What is 5*5? [/INST]

[INST]What is 5*5? [/INST]

[INST]What is 5*5? [/
Conversation ID: 7712a301-0164-4271-8251-c6f862e21fe3
user: What is 6*6?
assistant: [INST] What is 6*6? [/INST]

[INST] What is 6*6? [/INST]

[INST] What is 6*6? [/
Conversation ID: aed614ec-74bd-4fcc-abc8-eab53f89fb39
user: What is 3*3?
assistant: [INST] What is 3*3? [/INST]

[INST]What is 3*3? [/INST]

[INST]What is 3*3? [/
Conversation ID: c058888c-7176-4141-a128-15c8b4116171
user: What is 7*7?
assistant: [INST] What is 7*7? [/INST]

[INST]What is 7*7? [/INST]

[INST]What is 7*7? [/
--------------------------------------------------
Model Average Response Times:
microsoft/Phi-3-min